In [16]:
from networkx.algorithms.connectivity import local_node_connectivity
from networkx.algorithms.flow import shortest_augmenting_path
from networkx.algorithms.connectivity import minimum_st_node_cut
import networkx as nx
from networkx.algorithms import tree
from IPython.display import display, clear_output
from gerrychain import Partition
from gerrychain.grid import Grid
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import pickle
import math

In [17]:
def zdd_heuristic_recursion(G,s,t,origin_edge_dict,node_counter):
    assert(s != t)
    assert(s in G)
    assert(t in G)
    if len(G.nodes) == 2:
        # print('here')
        edge_lists = [origin_edge_dict[e] for e in G.edges()]
        return [i for el in edge_lists for i in el]
    cut_nodes = minimum_st_node_cut(G,s,t)
    G_ = G.copy()
    G_.remove_nodes_from(cut_nodes)
    CC = [G_.subgraph(c).copy() for c in nx.connected_components(G_)]
    assert(len(CC)==2)
    G0 = CC[0] if s in CC[0].nodes() else CC[1]
    G1 = CC[1] if t in CC[1].nodes() else CC[0]
    assert(s in G0.nodes())
    assert(t in G1.nodes())
    # print(cut_nodes, len(G0.nodes()),len(G1.nodes()),len(G.nodes()))
    G0.add_node('temp'+str(node_counter))
    G1.add_node('temp'+str(node_counter))
    neighbors_to_add0 = []
    neighbors_to_add1 = []
    origin_edge_dict0 = origin_edge_dict.copy()
    origin_edge_dict1 = origin_edge_dict.copy()
    for node in cut_nodes:
        for neighbor in G.neighbors(node):
            if neighbor in G0.nodes():
                G0.add_edge(neighbor,'temp'+str(node_counter))
                origin_edge_dict0[(neighbor,'temp'+str(node_counter))] = origin_edge_dict0[(neighbor,node)] + (origin_edge_dict0[(neighbor,'temp'+str(node_counter))] if (neighbor,'temp'+str(node_counter)) in origin_edge_dict0.keys() else [])
                origin_edge_dict0[('temp'+str(node_counter),neighbor)] = origin_edge_dict0[(neighbor,node)] + (origin_edge_dict0[('temp'+str(node_counter),neighbor)] if ('temp'+str(node_counter),neighbor) in origin_edge_dict0.keys() else [])
            elif neighbor in G1.nodes():
                G1.add_edge(neighbor,'temp'+str(node_counter))
                origin_edge_dict1[(neighbor,'temp'+str(node_counter))] = origin_edge_dict1[(neighbor,node)] + (origin_edge_dict1[(neighbor,'temp'+str(node_counter))] if (neighbor,'temp'+str(node_counter)) in origin_edge_dict1.keys() else [])
                origin_edge_dict1[('temp'+str(node_counter),neighbor)] = origin_edge_dict1[(neighbor,node)] + (origin_edge_dict1[('temp'+str(node_counter),neighbor)] if ('temp'+str(node_counter),neighbor) in origin_edge_dict1.keys() else [])
            else:
                print('neighbor error!')
                assert(False)
    return zdd_heuristic_recursion(G0,s,'temp'+str(node_counter),origin_edge_dict0,node_counter+1) + zdd_heuristic_recursion(G1,'temp'+str(node_counter),t,origin_edge_dict1,node_counter+1)

def zdd_heuristic_edge_order(G):
    #find farthest pair
    all_pairs_paths = dict(nx.all_pairs_shortest_path(G))
    path_lengths = {(u,v):len(all_pairs_paths[u][v]) for u in all_pairs_paths.keys() for v in all_pairs_paths[u]}
    s,t = random.choice([key for key in path_lengths.keys() if path_lengths[key]==max(path_lengths.values())])
    origin_edges = {(e[0],e[1]):[e] for e in G.edges()}
    origin_edges.update({(e[1],e[0]):[e] for e in G.edges()})
    return zdd_heuristic_recursion(G,s,t,origin_edges,0)

def frontiers(edge_list):
    frontier_list = [set()]
    edges_seen = []
    edges_unseen = edge_list
    for e in edge_list:
        edges_seen = edges_seen + [e]
        edges_unseen = edges_unseen[1:]
        frontier_list.append(({e[0] for e in edges_seen}.union({e[1] for e in edges_seen})).intersection({e[0] for e in edges_unseen}.union({e[1] for e in edges_unseen})))
    return frontier_list

def optimal_grid_edge_order(G):
    #assumes G is grid with (x,y) nodes
    x_max = max([v[0] for v in G.nodes()])
    y_max = max([v[1] for v in G.nodes()])
    edge_list = []
    for y in range(y_max):
        y_ = y+1
        x_ = 0
        while y_ > 0 and x_ < x_max:
            edge_list.append(((x_,y_),(x_,y_-1)))
            edge_list.append(((x_,y_-1),(x_+1,y_-1)))
            y_ -= 1
            x_ += 1
        if y_ > 0:
            edge_list.append(((x_,y_),(x_,y_-1)))
    for x in range(x_max):
        y_ = y_max
        x_ = x
        while y_ > 0 and x_ < x_max:
            edge_list.append(((x_,y_),(x_+1,y_)))
            edge_list.append(((x_+1,y_),(x_+1,y_-1)))
            y_ -= 1
            x_ += 1
        if x_ < x_max:
            edge_list.append(((x_,y_),(x_+1,y_)))
    assert(len(edge_list)==len(G.edges()))
    for e in edge_list:
        assert(e in G.edges())
    return edge_list

In [25]:
g = nx.grid_2d_graph(4,4)

In [28]:
g.edges()

EdgeView([((0, 0), (1, 0)), ((0, 0), (0, 1)), ((0, 1), (1, 1)), ((0, 1), (0, 2)), ((0, 2), (1, 2)), ((0, 2), (0, 3)), ((0, 3), (1, 3)), ((1, 0), (2, 0)), ((1, 0), (1, 1)), ((1, 1), (2, 1)), ((1, 1), (1, 2)), ((1, 2), (2, 2)), ((1, 2), (1, 3)), ((1, 3), (2, 3)), ((2, 0), (3, 0)), ((2, 0), (2, 1)), ((2, 1), (3, 1)), ((2, 1), (2, 2)), ((2, 2), (3, 2)), ((2, 2), (2, 3)), ((2, 3), (3, 3)), ((3, 0), (3, 1)), ((3, 1), (3, 2)), ((3, 2), (3, 3))])

In [23]:
#Testing

graph4 = nx.grid_2d_graph(4, 4) 
graph5 = nx.grid_2d_graph(5, 5)
graph6 = nx.grid_2d_graph(6,6)
graph10 = nx.grid_2d_graph(10, 10) 

graph = graph5
for i in [4, 5, 6, 10]:
    print(f"Creating an {i}x{i} graph...")
    graph = nx.grid_2d_graph(i,i)
    edge_list = zdd_heuristic_edge_order(graph)
    frontier_list = frontiers(edge_list)

    print(max([len(fron) for fron in frontier_list]),sum([len(fron) for fron in frontier_list]))
    print(max([len(fron) for fron in frontiers(list(graph.edges))]),sum([len(fron) for fron in frontiers(list(graph.edges))]))
    print(max([len(fron) for fron in frontiers(optimal_grid_edge_order(graph))]),sum([len(fron) for fron in frontiers(optimal_grid_edge_order(graph))]))

Creating an 4x4 graph...
5 83
5 89
4 76
Creating an 5x5 graph...
7 168
6 189
5 155
Creating an 6x6 graph...
8 305
7 344
6 274
Creating an 10x10 graph...
13 1486
11 1754
10 1310
